# Visualisations avancées

### Part 1: transformation donnée

In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
#Import library
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from dateutil.parser import parse
from scipy.interpolate import griddata
# import dash
# from dash import dcc
# from dash import html
# from dash.dependencies import Input, Output
from scipy.stats import gaussian_kde
import matplotlib.pyplot as plt
import io
import base64
from PIL import Image
import pandas as pd

In [19]:
# Liste des années pour lesquelles vous avez des fichiers CSV
annees = [2016, 2017,2018, 2019,2020,2021]

# Créez une liste pour stocker les DataFrames de chaque année
dataframes_par_annee = []

for annee in annees:
    # Chargez le fichier CSV de chaque année
    file_path = f"/Users/ceciliaacosta/IFT-DATASCIENCE/MILESTONE1_local/cleaned_data/{annee}.csv"  # Remplacez par le chemin réel
    df_annee = pd.read_csv(file_path)
    dataframes_par_annee.append(df_annee)

# Concaténez les DataFrames en un seul
nhl_data = pd.concat(dataframes_par_annee, ignore_index=True)

In [23]:
nhl_data_tr=nhl_data.copy()
nhl_data_tr['about.dateTime'] = nhl_data_tr['about.dateTime'].apply(lambda x: parse(x).year)

Index(['Unnamed: 0', 'about.period', 'about.dateTime', 'team.name',
       'result.eventTypeId', 'coordinates.x', 'coordinates.y',
       'result.secondaryType', 'result.emptyNet', 'result.strength.name',
       'player.fullName_Scorer', 'player.fullName_goalie', 'ID_game',
       'Game_type'],
      dtype='object')


In [21]:
from PIL import Image

# Open the image
image = Image.open('/Users/ceciliaacosta/IFT-DATASCIENCE/MILESTONE1_local/nhl_rink.png')
# Get the dimensions of the original image
width, height = image.size
# Calculate the coordinates for cropping
left = 0
top = 0
right = width / 2
bottom = height
# Crop the left half
left_half = image.crop((left, top, right, bottom))
left_half.save('left_half.png')

In [24]:
df = nhl_data_tr.copy() 
condition = df['coordinates.x'] < 0 
df.loc[condition, ['coordinates.x', 'coordinates.y']] *= -1
df['coordinates.x'] = abs(df['coordinates.x'] - 89)

### Step two : calculate the compute aggregate statistics of shot locations across the entire league to calculate the league average shot rate per hour

In [47]:
# Define the rink dimensions 
rink_width_feet = 85  # Width of the rink in feet
rink_length_feet = 200  # Length of the rink in feet

def shotrate_per_location(df):
    # Calculate the league average shot rate per hour across all locations
    total_games = df['ID_game'].nunique()

    # Total shots per location 
    ser_total_shots= df.groupby(['coordinates.x', 'coordinates.y']).count()
    df_total_shots=pd.DataFrame(ser_total_shots['ID_game'])
    df_total_shots=df_total_shots.rename(columns={'ID_game':'coord_total_shots'})

    # Calculate the rate per hour of the shots per location
    df_rate_shots=df_total_shots.copy()
    df_rate_shots['coord_total_shots'] = df_rate_shots['coord_total_shots'].apply(lambda x: x / total_games) # We assume a game is 1 hour long
    # We assume a game is 1 hour long
    df_rate_shots.rename(columns={'coord_total_shots':'coord_rate_shots'})

    return df_rate_shots


coord_rate_shots
coordinates.x coordinates.y                  
0.0           -40.0                  0.000132
              -39.0                  0.000132
              -38.0                  0.000132
              -37.0                  0.000263
              -36.0                  0.000132
...                                       ...
89.0           38.0                  0.001053
               39.0                  0.001317
               40.0                  0.000922
               41.0                  0.000527
               42.0                  0.000132

[7483 rows x 1 columns]

### Compute the excess rate for a specific team and season:

In [ ]:
def excess_for_team(teamdf:pd.DataFrame, acrossteam:pd.DataFrame):
    """
    This function calculates the excess of shots for a team in a given location
    """
    teamdf=teamdf.copy()
    acrossteam=acrossteam.copy()
    teamdf=shotrate_per_location(teamdf) # Only shots for a team
    acrossteam=shotrate_per_location(acrossteam) # Shots accross all teams

    # Calculate the excess of shots for a team in a given location
    teamdf['excess_shots']=teamdf['coord_rate_shots']-acrossteam['coord_rate_shots']
    return teamdf

pass

In [18]:
from scipy.ndimage import gaussian_filter
from matplotlib import image as mpimg

# Create a KDE (Kernel Density Estimation) for shot data using a Gaussian kernel
shot_coordinates = np.vstack([np.abs(shot_data['coordinates.x']), shot_data['coordinates.y']])

# Create a mesh grid for plotting
x, y = np.meshgrid(np.linspace(0,90,91),
                   np.linspace(-42.5,42.5,81))
# Evaluate the KDE on the mesh grid to get the estimated density values

positions = np.vstack([x.ravel(), y.ravel()])

z=griddata((np.abs(shot_data['coordinates.x']), shot_data['coordinates.y']),, positions, method='cubic', fill_value=0)
z=pd.DataFrame(z)*100
zsmooth=gaussian_filter(z, sigma=6)


rink_image = mpimg.imread('left_half.png')  # Replace with the path to your image
fig, ax = plt.subplots()
# Display the rink image in the background
ax.imshow(rink_image, origin='lower', extent=[0, 100, -21.25, 21.25])
contour = ax.contourf(x, y, zsmooth, cmap='viridis', levels=20, alpha=0.7, origin='lower', cmin=0, cmax=merged_data['excess_shot_rate'].max())
cbar = plt.colorbar(contour, ax=ax, orientation='vertical', label='Excess Shot Rate')
plt.title('Shot Density Map')
plt.show()


ValueError: different number of values and points

# Test widget

In [ ]:
with open('left_half.png', "rb") as image_file:
    encoded_image = base64.b64encode(image_file.read()).decode("utf-8")
img = Image.open(io.BytesIO(base64.b64decode(encoded_image)))
ommg=img.rotate(90)
ommg.save('nhl_rinkk.png')


In [ ]:
img = Image.open('nhl_rinkk.png')
width, height = img.size
print(f'Image dimensions: {width}x{height}')

Image dimensions: 550x467


In [ ]:
import matplotlib.pyplot as plt

data=nhl_data_tr.copy()
# Sample ice rink image (replace with your image)
ice_rink_image_path = 'nhl_rinkk.png'  # Replace with your ice rink image

# Load the ice rink image
with open(ice_rink_image_path, "rb") as image_file:
    encoded_image = base64.b64encode(image_file.read()).decode("utf-8")

# Create a function to generate the plot
def generate_plot(selected_team, selected_season, selected_year):
    team_filtered_data = data[(data['team.name'] == selected_team) &
                         (data['Game_type'] == selected_season) &
                         (data['about.dateTime'] == selected_year)]
    
    accrossteam_filtered_data = data[(data['Game_type'] == selected_season) & (data['about.dateTime'] == selected_year)]

    
    # Create a scatter plot
    # plt.figure()
    
    # plt.scatter(filtered_data['coordinates.x'], filtered_data['coordinates.y'], c='red', marker='o', label='Shots')
    # plt.legend()
    # plt.title('Shot Map by Season')
    # plt.xlabel('X Coordinate')
    # plt.ylabel('Y Coordinate')
    #plt.xlim(0, 90)  # Adjust the limits based on your data
    #plt.ylim(-3.4, 100 - 3.4)   # Adjust the limits based on your data
    
    # Load the ice rink image as the background
    img = Image.open(io.BytesIO(base64.b64decode(encoded_image)))
    
    plt.imshow(img, extent=[0, 100, 0, 90], zorder=0)

    # Save the plot as a temporary image
    output_buffer = io.BytesIO()
    plt.savefig(output_buffer, format="png")
    output_buffer.seek(0)

    return output_buffer

# Create a Dash application
app = dash.Dash(__name__)
@app.callback(
    Output('shot-map', 'src'),
    [Input('team-dropdown', 'value'), Input('season-dropdown', 'value'), Input('year-dropdown', 'value')]
)
def update_plot(selected_team, selected_season,selected_year):
    output_buffer = generate_plot(selected_team, selected_season,selected_year)
    return f"data:image/png;base64,{base64.b64encode(output_buffer.read()).decode('utf-8')}"

# Create dropdown menus for team, season, and year
team_dropdown = dcc.Dropdown(
    id='team-dropdown',
    options=[{'label': team, 'value': team} for team in data['team.name'].unique()]
)

season_dropdown = dcc.Dropdown(
    id='season-dropdown',
    options=[{'label': season, 'value': season} for season in data['Game_type'].unique()],
    value=data['Game_type'].unique()[0]
)
year_dropdown = dcc.Dropdown(
    id='year-dropdown',
    options=[{'label': year, 'value': year} for year in data['about.dateTime'].unique()],
    value=data['about.dateTime'].unique()[0]
)


# Create the initial layout with dropdowns and the image
app.layout = html.Div([
    html.H1('NHL Shot Map'),
    team_dropdown,
    season_dropdown,
    year_dropdown,
    html.Img(id='shot-map')
])

if __name__ == '__main__':
    app.run_server(debug=True)
